# Predict

In [9]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None):
    obs = [np.zeros([4, 8, 2]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return obs

In [10]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card 
        
    return None

In [39]:
import itertools
calc_correct_output_sample_cache = {}

In [49]:
def set_env(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None):
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    if opponent_hand_cards is None:
        env.players[1].hand_cards = []
        for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
            env.players[1].hand_cards.append(env.cards_left.pop())      
    else:        
        for card in opponent_hand_cards:
            env.cards_left.remove(card)
        env.players[1].hand_cards = opponent_hand_cards[:]
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
        
def generate_hand_cards_key(hand_cards):
    card_ids = []
    for card in hand_cards:
        card_ids.append(card.id)
    card_ids.sort()
    return str(card_ids)

def generate_cache_key(hand_cards, opponent_hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    return generate_hand_cards_key(hand_cards) + "-" + generate_hand_cards_key(opponent_hand_cards) + "-" + (str(table_card.id) if table_card is not None else "") + "-" + str(own_tricks) + "-" + str(opponent_tricks)
    
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None):
   
    cache_key = generate_cache_key(hand_cards, opponent_hand_cards, table_card, own_tricks, opponent_tricks)
    if not cache_key in calc_correct_output_sample_cache:
    
        set_env(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards)

        #root = State(1, env.get_state(), env.current_player)
        #a, p = mcts_game_step(root, 100)
       # env.set_state(root.env_state)
       # print(p)
        #draw_tree(root)
        #print("")
        #draw_path(root)
        #obs, rew, is_done, _ = env.step(hand_cards[0].id)
       # print(rew, is_done)

        #env.render('human')
        #root = State(1, env.get_state())
        #a, p = mcts_game_step(root, 1000, False)
        #draw_tree(root)
        #print("")
        #draw_tree(root.childs[0])

        #draw_path(root)
        #reset_samples(1)
        correct_output = search(env.regenerate_obs())[1]
        #postprocess_samples()
        
        
        calc_correct_output_sample_cache[cache_key] = {}
        i = 0
        for card in hand_cards:
            calc_correct_output_sample_cache[cache_key][card.id] = correct_output[i]
            i += 1
            
    output = np.zeros((len(hand_cards,)))
    i = 0
    for card in hand_cards:
        output[i] = calc_correct_output_sample_cache[cache_key][card.id]
        i += 1
        
    return output#sample_outputs[0]

def show_mcts(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None, tree_depth=5, tree_path=[], steps=100, debug=False, use_model=True):
    set_env(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards)
    if debug:
        env.render('human')
    root = State(1, env.get_state(), env.current_player)
    a, p = mcts_game_step(root, steps, use_model=use_model)
    env.set_state(root.env_state)
    if debug:
        draw_tree(root, tree_depth, tree_path)
    return p

In [43]:

def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, possible_hand_cards=[]):    

    correct_output = None    
    n = 0
    for opposite_hand_cards in possible_hand_cards:
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks, list(opposite_hand_cards))

        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs 

        n += 1

    return correct_output / n

In [14]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict_single(input)[0]
    correct_output = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output,correct_output)
    id = 0
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[id]) + ")")
        id += 1

In [15]:
def generate_random_game(only_initial_state=False):
    if only_initial_state:
        own_tricks = 0
        opponent_tricks = 0
        table_card_enabled = 0
    else:
        own_tricks = random.randint(0, 1)
        opponent_tricks = random.randint(0, 1)
        if opponent_tricks == 0 and own_tricks == 1:
            table_card_enabled = 0
        elif opponent_tricks == 1 and own_tricks == 0:
            table_card_enabled = 1
        else:        
            table_card_enabled = random.randint(0, 1)
        
    hand_cards = random.sample(env.cards, (3 - own_tricks - opponent_tricks) * 2)
    table_card = hand_cards.pop() if table_card_enabled else None
    
    opponent_hand_cards = []
    for i in range(len(hand_cards) // 2):
        opponent_hand_cards.append(hand_cards.pop())
        
    return hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards

def generate_game_from_obs(obs):
    own_tricks = 0
    if obs[1][0]:
        own_tricks += 1
    if obs[1][1]:
        own_tricks += 2   
        
    opponent_tricks = 0
    if obs[1][2]:
        opponent_tricks += 1
    if obs[1][3]:
        opponent_tricks += 2    
        
    cards_left = env.cards[:]
    random.shuffle(cards_left)
    hand_cards = []
    table_card = None
    for c in Color:
        for v in Value: 
            if obs[0][c.value][v.value][0]:
                hand_cards.append(card(c, v))
                cards_left.remove(hand_cards[-1])
            if obs[0][c.value][v.value][1]:
                table_card = card(c, v)
                cards_left.remove(table_card)
    
    opponent_hand_cards = []
    for i in range(3 - own_tricks - opponent_tricks - (1 if table_card is not None else 0)):
        opponent_hand_cards.append(cards_left.pop())
        
    return hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards

def create_eval_cache():
    eval_cache_input = []
    eval_cache_output = []
    eval_cache_initial_states = []
    for i in range(200):
        game = generate_random_game(True)
        set_env(*game)
        obs, is_done = env.regenerate_obs(), False
        eval_cache_initial_states.append(len(eval_cache_input))
        while not is_done:
            game = generate_game_from_obs(obs)
            hand_cards = game[0]
            eval_cache_input.append([np.array(obs[0]), np.array(obs[1])])
            state = env.get_state()
            output = calc_correct_output(*game[:4])
            env.set_state(state)

            outputs_per_card = np.zeros((32,))
            for i in range(len(hand_cards)):
                outputs_per_card[hand_cards[i].id] = output[i]

            eval_cache_output.append(outputs_per_card)
            
            obs, _, is_done, _ = env.step(np.argmax(outputs_per_card))
        
    return eval_cache_input, eval_cache_output, eval_cache_initial_states
        
def eval2(model, eval_cache_input, eval_cache_output, debug=False):
    correct = 0
    for i in range(len(eval_cache_input)):

        output = model.predict_single(eval_cache_input[i])[0]
        correct_output = eval_cache_output[i]
                    
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        if debug and correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
            key = model.generate_key(eval_cache_input[i])
            print(i, correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)],  model.table[key][2] if key in model.table else 0)

    return correct / len(eval_cache_input)

def eval_game(model):    
    current_player = env.current_player
    if env.current_player == 1:
        state = env.get_state()
        n = len(env.players[env.current_player].hand_cards)   
        p = 0
        distr = np.zeros((n,))

        for i in range(n):
            card_id = env.players[env.current_player].hand_cards[i].id
            obs, rew, is_done, _ = env.step(card_id)

            if is_done:
                distr[i] = (rew[0] > 0)
            else:
                distr[i] = ((1 - eval_game(model)) if current_player != env.current_player else eval_game(model))
            env.set_state(state)

        return distr.mean()
    else:
        if model is None:
            state = env.get_state()
            p = calc_correct_output_sample(env.players[env.current_player].hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks, env.players[1 - env.current_player].hand_cards)
            env.set_state(state)
            step = env.players[env.current_player].hand_cards[np.argmax(p)].id
        else:
            p, v = model.predict_single(env.regenerate_obs())            
  
            valid_moves = np.zeros((32,))       
            for card in env.players[env.current_player].hand_cards:
                valid_moves[card.id] = 1

            prediction_valid = np.ma.masked_where(valid_moves == 0, p)
            step = np.argmax(prediction_valid)      

        obs, rew, is_done, _ = env.step(step)
        if is_done:
            return (rew[0] > 0)
        else:
            return ((1 - eval_game(model)) if current_player != env.current_player else eval_game(model))

def define_eval_games():
    eval_games = []
    hand_card_combinations = itertools.combinations(env.cards[:], 3)
    for hand_cards in hand_card_combinations:
        cards_left = env.cards[:]
        for card in hand_cards:
            cards_left.remove(card)
            
        other_hand_card_combinations = itertools.combinations(cards_left, 3)
        for other_hand_cards in other_hand_card_combinations:
            eval_games.append((list(hand_cards), None, 0, 0, list(other_hand_cards)))
        
    return eval_games
        
def eval(model, eval_games):
    correct = 0
    for game in eval_games:
        for start_player in range(2):
            set_env(*game)
            env.current_player = start_player
            correct += ((1 - eval_game(model)) if start_player == 1 else eval_game(model))  

    return correct / (len(eval_games) * 2)

In [54]:
def calc_exploitability_in_game(model, possible_hand_cards):
    current_player = env.current_player
    if env.current_player == 1:
        state = env.get_state()
        p = calc_correct_output(env.players[env.current_player].hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks, possible_hand_cards)
        env.set_state(state)
        step = env.players[env.current_player].hand_cards[np.argmax(p)].id
    else:
        p, v = model.predict_single(env.regenerate_obs())            

        valid_moves = np.zeros((32,))       
        for card in env.players[env.current_player].hand_cards:
            valid_moves[card.id] = 1

        prediction_valid = np.ma.masked_where(valid_moves == 0, p)
        step = np.argmax(prediction_valid)      
        
        possible_next_hand_cards = []
        for hand_cards in possible_hand_cards:
            p, v = model.predict_single(generate_input(hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks))
            
            valid_moves = np.zeros((32,))       
            for card in hand_cards:
                valid_moves[card.id] = 1

            prediction_valid = np.ma.masked_where(valid_moves == 0, p)
            theoretical_step = np.argmax(prediction_valid)      
            if step == theoretical_step:
                possible_next_hand_cards.append(hand_cards[:])
                possible_next_hand_cards[-1].remove(env.cards[step])
        possible_hand_cards = possible_next_hand_cards

    obs, rew, is_done, _ = env.step(step)
    if is_done:
        return (rew[0] > 0)
    else:
        return ((1 - calc_exploitability_in_game(model, possible_hand_cards)) if current_player != env.current_player else calc_exploitability_in_game(model, possible_hand_cards))
    
def calc_exploitability(model, eval_games):
    global calc_correct_output_sample_cache
    exploitability = 0
    calc_correct_output_sample_cache = {}
    for game in eval_games:
        for start_player in range(2):
            set_env(*game)
            env.current_player = start_player

            possible_opposite_cards = env.cards[:]
            for card in env.players[1].hand_cards:
                possible_opposite_cards.remove(card)
                
            opposite_hand_card_combinations = itertools.combinations(possible_opposite_cards, len(env.players[0].hand_cards))
            opposite_hand_card_combinations = [list(hand_cards) for hand_cards in opposite_hand_card_combinations]
            
            winner = ((1 - calc_exploitability_in_game(model, opposite_hand_card_combinations)) if start_player == 1 else calc_exploitability_in_game(model, opposite_hand_card_combinations))  
            exploitability += (-1 if winner == 1 else 1)

    return exploitability / (len(eval_games) * 2)

In [30]:
eval_cache_input, eval_cache_output, eval_cache_initial_states = create_eval_cache()

In [73]:
len(calc_correct_output_cache)

596

In [82]:
compare(None, None)

0.4964

In [166]:
with open('eval_cache_2s3k', 'wb') as fp:
    pickle.dump([eval_cache_input, eval_cache_output], fp)

In [21]:
with open ('eval_cache_2s3k', 'rb') as fp:
    eval_cache_input, eval_cache_output = pickle.load(fp)

In [17]:
eval_games = define_eval_games()
len(eval_games)

560

In [100]:
calc_exploitability(best_model, eval_games)

0.019642857142857142

In [355]:
env.reset()
env.render('human')
state = env.get_state()

In [377]:
env.set_state(state)
eval_game(None)

0.0

In [423]:
set_env(*eval_games[0])
eval_game(model)

1.0

In [116]:
for i in range(56):
    print(i, eval(None, eval_games[i * 10:(i+1) * 10]), eval(model, eval_games[i * 10:(i+1) * 10]))

0 0.5 0.5
1 0.5 0.5
2 0.5 0.5
3 0.4499999999999999 0.4499999999999999
4 0.5 0.5
5 0.5 0.5
6 0.4 0.4
7 0.5 0.5
8 0.4499999999999999 0.4499999999999999
9 0.39999999999999997 0.4333333333333334
10 0.3500000000000001 0.4
11 0.5 0.5
12 0.4333333333333333 0.4000000000000001
13 0.36666666666666664 0.36666666666666664
14 0.30000000000000004 0.30000000000000004
15 0.5 0.5
16 0.5 0.5
17 0.5 0.5
18 0.5 0.5
19 0.45 0.45
20 0.39999999999999997 0.39999999999999997
21 0.3 0.3
22 0.5 0.5
23 0.4166666666666667 0.4083333333333333
24 0.35833333333333334 0.35833333333333334
25 0.3 0.3
26 0.45 0.45
27 0.3583333333333334 0.36666666666666664
28 0.2833333333333333 0.29166666666666663
29 0.22500000000000003 0.22500000000000003
30 0.45 0.5
31 0.425 0.4666666666666667
32 0.4 0.4333333333333334
33 0.4083333333333334 0.4166666666666667
34 0.3583333333333334 0.35833333333333334
35 0.3166666666666667 0.3166666666666667
36 0.4 0.4
37 0.3166666666666667 0.3166666666666667
38 0.2166666666666667 0.23333333333333334
39 0

In [172]:
eval(model, eval_games)

0.6874999999999999

In [213]:
eval(None, eval_games)

0.7303571428571419

In [443]:
eval(model, eval_cache_input, eval_cache_output, debug=True)

TypeError: eval() got an unexpected keyword argument 'debug'

In [211]:
eval_cache_input[212]

[array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]]), array([0., 0., 0., 0.])]

In [212]:
model.table[model.generate_key(eval_cache_input[212])]

[array([266.77957282,   0.        ,   0.        , 261.46378714,
          0.        ,   0.        ,   0.        , 272.75664004,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]),
 array([-275.]),
 801]

In [213]:
eval_cache_output[212]

array([0.1, 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. ])

In [180]:
predict(*generate_game_from_obs(eval_cache_input[354])[:4])

[0.         0.25238272 0.25316138 0.         0.4944559  0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ] [1. 1. 1.]
Color.EICHEL Value.KOENIG => 0.25238271995816913 (1.0)
Color.EICHEL Value.OBER => 0.2531613804182263 (1.0)
Color.GRUEN Value.SAU => 0.4944558996236046 (1.0)


In [181]:
calc_correct_output(*generate_game_from_obs(eval_cache_input[354])[:4])

array([1. , 1. , 0.4])

In [262]:
game = generate_game_from_obs([sample_inputs[0][100],sample_inputs[1][100]])
#game = (game[0], game[1], game[2], game[3], [card(Color.HERZ, Value.NEUN), card(Color.SCHELLN, Value.SIEBEN), card(Color.HERZ, Value.UNTER), card(Color.GRUEN, Value.KOENIG), card(Color.GRUEN, Value.SAU)])
calc_correct_output_sample(*game)

array([0.66666667, 0.66666667, 0.33333333])

In [263]:
show_mcts(*game, debug=True, steps=20, use_model=True, tree_path=[], tree_depth=3)

FigureCanvasNbAgg()

array([0.21937792, 0.54154378, 0.2390783 ])

In [171]:
k = model.generate_key(generate_input([card(Color.HERZ, Value.ZEHN), card(Color.GRUEN, Value.ACHT), card(Color.EICHEL, Value.OBER)], card(Color.EICHEL, Value.NEUN), 0, 0))
model.table[k]

[array([0.        , 0.        , 1.34194389, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.83304733,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.82500878, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]), array([1.]), 3]

In [218]:
val = []
for i in range(1000):
    val.append(show_mcts(*generate_game_from_obs(eval_cache_input[212]), steps=20, use_model=True))
np.mean(val, 0)


array([0.3374612, 0.3250058, 0.337533 ])

In [113]:
model.predict_single(generate_input([card(Color.EICHEL, Value.NEUN), card(Color.HERZ, Value.ZEHN)], card(Color.GRUEN, Value.OBER), 2, 1))

[array([4.5539989e-04, 6.4499147e-04, 6.2959566e-04, 5.4278685e-04,
        1.4150019e-03, 4.5420015e-01, 1.5384635e-03, 9.1649400e-04,
        1.7755200e-03, 1.5444498e-03, 8.8261435e-04, 7.3095079e-04,
        5.2650593e-04, 9.9580095e-04, 1.1090455e-03, 6.0478464e-04,
        8.7860849e-04, 2.2552728e-03, 1.6426475e-03, 1.5226839e-03,
        5.6466490e-01, 2.0387939e-03, 2.4349622e-03, 1.9592713e-03,
        1.3109865e-03, 4.4825173e-04, 5.8681599e-04, 3.4538191e-04,
        1.2316451e-03, 8.4385881e-04, 1.8078578e-03, 1.2765919e-03],
       dtype=float32), array([0.46111563], dtype=float32)]

In [285]:
calc_correct_output_sample([
            card(Color.GRUEN, Value.SIEBEN), 
            card(Color.GRUEN, Value.OBER), 
            card(Color.SCHELLN, Value.NEUN)
        ], None, 2, 0)

[0.23458585 0.21142988 0.55398427]


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

array([0., 0., 1.])

In [255]:
predict([
            card(Color.GRUEN, Value.SIEBEN), 
            card(Color.GRUEN, Value.KOENIG),
            card(Color.HERZ, Value.KOENIG)
        ], card(Color.EICHEL, Value.ZEHN), 1, 1)

[0.39571235 0.33549584 0.26879181]


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

[0.00219368 0.00295891 0.00492629 0.00228125 0.00067742 0.0019176
 0.00160495 0.00306738 0.00170262 0.2736106  0.00104753 0.00065809
 0.00124946 0.00122479 0.00083221 0.34336203 0.00157421 0.35396522
 0.0022292  0.00107221 0.0017584  0.00199055 0.0022874  0.00158446
 0.00120892 0.00238398 0.00235519 0.00170699 0.0021641  0.00231255
 0.00093804 0.00114301] [0. 0. 0.]
Color.GRUEN Value.SIEBEN => 0.34336203 (0.0)
Color.GRUEN Value.KOENIG => 0.2736106 (0.0)
Color.HERZ Value.KOENIG => 0.35396522 (0.0)


In [257]:
predict([
    card(Color.EICHEL, Value.UNTER), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],card(Color.GRUEN, Value.OBER), 0, 0)

[0.16209546 0.29126847 0.14643211 0.25181529 0.14838867]


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

[0.00225554 0.12882802 0.00333104 0.16498454 0.00407682 0.00374047
 0.00184983 0.01050379 0.00643561 0.36693758 0.00278587 0.00835195
 0.22336954 0.00493929 0.00303816 0.00170397 0.00097518 0.0054345
 0.00415214 0.00438484 0.00151944 0.00706366 0.00218979 0.21273391
 0.0010833  0.00248363 0.00400397 0.00299505 0.00209963 0.00448016
 0.00377667 0.00315689] [0. 1. 0. 1. 1.]
Color.EICHEL Value.UNTER => 0.16498454 (0.0)
Color.GRUEN Value.KOENIG => 0.36693758 (1.0)
Color.EICHEL Value.KOENIG => 0.12882802 (0.0)
Color.HERZ Value.SIEBEN => 0.21273391 (1.0)
Color.GRUEN Value.ZEHN => 0.22336954 (1.0)


In [261]:
predict([
    card(Color.HERZ, Value.SIEBEN), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.HERZ, Value.ACHT),
    card(Color.EICHEL, Value.SAU),
    card(Color.GRUEN, Value.UNTER),
    ],card(Color.SCHELLN, Value.OBER), 0, 0)

[0.28631919 0.15695597 0.25385176 0.14562401 0.15724907]


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

[0.21805565 0.00234719 0.00126939 0.00292074 0.00098915 0.00168285
 0.00176553 0.00741228 0.00306193 0.20693102 0.00235981 0.17444488
 0.00192268 0.00234325 0.00263577 0.00079589 0.00064911 0.00227424
 0.00357403 0.00439737 0.00371622 0.00286246 0.19036686 0.19911051
 0.00340857 0.0075426  0.0011064  0.00157739 0.00197756 0.00082587
 0.00150106 0.00150635] [0. 0. 0. 0. 0.]
Color.HERZ Value.SIEBEN => 0.19911051 (0.0)
Color.GRUEN Value.KOENIG => 0.20693102 (0.0)
Color.HERZ Value.ACHT => 0.19036686 (0.0)
Color.EICHEL Value.SAU => 0.21805565 (0.0)
Color.GRUEN Value.UNTER => 0.17444488 (0.0)


In [395]:
hard_input = generate_input([
    card(Color.EICHEL, Value.UNTER), 
    #card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],None, 1, 0)
model.predict_single(hard_input)

[array([3.2768894e-03, 1.9878964e-01, 1.3820220e-02, 4.6424335e-01,
        7.1053780e-03, 2.9274071e-02, 8.1222923e-03, 5.4989127e-04,
        8.0019347e-03, 4.3923180e-03, 4.0565468e-03, 3.2476720e-03,
        2.0850727e-01, 4.3509626e-03, 5.9748022e-03, 6.1062318e-03,
        1.3822509e-03, 2.1725877e-04, 1.3098384e-03, 7.1340741e-04,
        2.0487478e-03, 3.3812402e-03, 1.0878793e-04, 7.3019944e-02,
        3.4260689e-03, 2.7179192e-03, 1.3691070e-03, 6.1381124e-03,
        4.5132274e-03, 4.4453251e-03, 2.2470709e-03, 4.3023235e-04],
       dtype=float32), array([0.5526783], dtype=float32)]

In [ ]:
for sample_input in sample_inputs[0]:
    if np.array_equal(sample_input,hard_input[0]):
        print("test")

In [80]:
compare(model, None)

0.48

In [72]:
match([model, None], True)

[0. 0. 1. 0. 0.]
[1. 1. 0. 1.]
[1. 1. 0.]
[1. 0.]
[1.]


1